In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jun  4 12:22:27 2019

@author: abhalla2
"""

!pip install -U Contractions
import nltk
nltk.download('all')


#Importing various libraries
import pandas as pd
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
import matplotlib.pyplot as plt
import re, string, unicodedata
import nltk
#import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Bidirectional
import keras 
from keras.layers import LSTM, CuDNNLSTM
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.utils import to_categorical



#Ignoring the warnings
import warnings
warnings.filterwarnings(action = 'ignore') 


#Reading the data
df2 = pd.read_csv('../input/training/data_combined_train.csv')
X = df2.iloc[:, 0:1].values
X_train = []
for i in X:
    for j in i:
        X_train.append(str(j))
y_train = df2.iloc[:, 1].values

def replace_contractions(sentence):
    """Replace contractions in string of text"""
    return contractions.fix(sentence)

  
def words_list(sample):
    words = []
    """Tokenising the corpus"""
    for i in sample: 
        temp = []
        for j in word_tokenize(i):
            temp.append(j.lower())
        temp = normalize(temp)
        words.append(temp)
    return words

  
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

  
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

  
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

"""def replace_numbers(words):
    Replace all interger occurrences in list of tokenized words with textual representation
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words
"""


def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

  
def lemmatize_words(words):
    """Lemmatize the words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word)
        lemmas.append(lemma)
    return lemmas

 
def normalize(words):
    """This is the main function which takes all other functions to pre-process the data given"""
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    lemmatize_words(words)
    return words


vocab_size = 100000
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 130
batch_size = 32
word_list = words_list(X_train)
X_train = []
for list_of_words in word_list:
    sentence = ' '.join(x for x in list_of_words)
    X_train.append(sentence)
X_train, y_train = shuffle(X_train, y_train)
t = Tokenizer()
t.fit_on_texts(X_train)
encoded_word_list = t.texts_to_sequences(X_train)
X_train = pad_sequences(encoded_word_list, maxlen=maxlen, padding='post')


#Managing the y vector
y_train = to_categorical(y_train, num_classes = 10)
print(y_train[0])

#Regularization
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

#Constructing and training the model
model = Sequential()
e = Embedding(input_dim = vocab_size, output_dim = 100, input_length = X_train.shape[1], dropout=0.2)  #130*100 Output
model.add(e)
tensors = model.add(Bidirectional(LSTM(128, return_sequences = False, activation = 'tanh')))
model.add(Dropout(0.2))
model.add(Dense(10, activation = 'softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=3, batch_size=batch_size, verbose=1, validation_split=0.1, callbacks=[es, mc])

# load the saved model
#saved_model = load_model('best_model.h5')

#Plotting the results
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

#Testing
dataset2 = pd.read_csv('../input/training/data_combined_train.csv')
X_test_init = dataset2.iloc[:, 0:1].values
y_test = dataset2.iloc[:, 1].values
X_test = []
for i in X_test_init:
    for j in i:
        X_test.append(str(j))
word_list = words_list(X_test)
X_test = []
for list_of_words in word_list:
    sentence = ' '.join(x for x in list_of_words)
    X_test.append(sentence)
encoded_word_list = t.texts_to_sequences(X_test)
X_test = pad_sequences(encoded_word_list, maxlen=maxlen, padding='post')
#Managing the y vector
y_test = to_categorical(y_test, num_classes = 10)
score, acc = model.evaluate(X_test, y_test, batch_size = batch_size, verbose = 1)
print("The accuracy of the model on the test set is: ", acc, score)

#Confusion Matrix
y_pred = model.predict(X_test)
y_pred = [ np.argmax(t) for t in y_pred ]
y_test_non_category = [ np.argmax(t) for t in y_test]
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test_non_category, y_pred)
print(conf_mat)

ERROR: Could not find a version that satisfies the requirement Contractions (from versions: none)
ERROR: No matching distribution found for Contractions
[nltk_data] Error loading all: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


ModuleNotFoundError: No module named 'contractions'